## 실시간 데이터 처리 개념과 소켓 열기
* 실시간 데이터 스트리밍을 위한 네트워크 소켓을 설정하여 데이터를 Spark에 전달할 수 있도록 합니다. 이 예제를 통해 소켓을 설정할 수 있습니다.

### Netcat을 이용한 소켓 열기
* 다음 명령어를 사용하여 로컬 호스트의 9999 포트에서 데이터를 지속적으로 수신합니다.

## Netcat 설치
* sudo apt update
* sudo apt install netcat

## 소켓 열기
* nc -lk 9999


# 실시간 음식 검색 집계 프로젝트

배달 앱의 데이터 엔지니어로서, 사용자가 음식 이름을 검색할 때마다 특정 음식에 대한 검색 빈도를 실시간으로 집계하고 모니터링하는 시스템을 구축합니다.

## 요구사항

다음 조건을 만족하는 Spark Structured Streaming 애플리케이션을 작성하세요.

### 1. 실시간 음식 검색 집계 프로그램
- Spark Structured Streaming을 사용하여 특정 음식 이름의 검색 빈도를 실시간으로 집계합니다.

### 2. 특정 음식 필터링 조건 설정
- 사용자가 검색한 음식 중 '피자', '치킨', '버거'에 대한 검색만 집계합니다.
- 이 외의 음식 검색은 무시합니다.

### 3. 데이터 수신 방식
- 소켓 스트림을 통해 실시간 데이터를 수신합니다.
- `nc -lk 9999` 명령어로 소켓 서버를 열고, Spark 애플리케이션에서 localhost:9999 포트를 통해 데이터를 수신합니다. (리눅스 기준)
- 예시: 터미널에서 `nc -lk 9999` 명령어를 입력한 후, 텍스트로 음식 이름을 입력하여 Spark가 이를 실시간으로 수신합니다.

### 4. 타임스탬프와 윈도우 설정
- 타임스탬프 추가: 각 데이터 항목에 현재 시간 타임스탬프를 추가하여 이벤트 시간 기반 처리가 가능하도록 합니다.
- 워터마크 설정: 10분이 지난 데이터는 지연된 데이터로 간주하여 집계에서 제외합니다.
- 1분 고정 윈도우: 각 음식별 검색 빈도를 1분 단위로 집계합니다. 즉, 매 1분마다 음식별 검색 횟수를 집계합니다.

### 5. 출력 방식
- 매 트리거(5초)마다 현재 윈도우의 모든 검색 빈도를 콘솔에 출력합니다.
- outputMode("complete")을 사용하여, 윈도우의 전체 집계 결과를 매번 콘솔에 출력합니다.

## 입력 예시
1. 새로운 터미널 창을 열고 `nc -lk 9999` 명령어로 소켓 서버를 실행합니다.
2. 소켓 서버가 실행 중인 터미널에 아래와 같은 데이터를 입력합니다.

   ```plaintext
   피자 치킨 버거
   피자
   치킨 치킨 버거
   버거 피자

## 출력 
- `food` 열은 사용자가 검색한 음식 이름입니다. 설정한 필터링 조건에 따라 '피자', '치킨', '버거'가 포함됩니다.
- `window` 열은 1분 간격으로 집계된 윈도우 간격을 나타냅니다. 
   - 예를 들어, `{2024-11-07 08:05:00, 2024-11-07 08:06:00}`은 08:05:00부터 08:06:00까지 1분간의 데이터를 포함합니다.
- `count` 열은 해당 윈도우 간격 동안 특정 음식이 검색된 횟수를 나타냅니다.

| window                                     | food   | count |
|--------------------------------------------|--------|-------|
| {2024-11-07 08:05:00, 2024-11-07 08:06:00} | 피자   | 1     |
| {2024-11-07 08:05:00, 2024-11-07 08:06:00} | 치킨   | 1     |
| {2024-11-07 08:06:00, 2024-11-07 08:07:00} | 버거   | 1     |
| {2024-11-07 08:06:00, 2024-11-07 08:07:00} | 피자   | 1     |
| {2024-11-07 08:07:00, 2024-11-07 08:08:00} | 치킨   | 2     |
| {2024-11-07 08:07:00, 2024-11-07 08:08:00} | 버거   | 1     |
| {2024-11-07 08:08:00, 2024-11-07 08:09:00} | 버거   | 1     |
| {2024-11-07 08:08:00, 2024-11-07 08:09:00} | 피자   | 1     |



### TODO: 주석과 참고 페이지를 참고하여 실시간 스트리밍 데이터 처리를 가능하도록 완성해주세요.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, current_timestamp, window, expr, col

# 기존 Spark 세션 종료 후 새로운 세션 생성
# (Spark 세션이 이미 존재할 경우 중지하고, 새로운 세션을 시작)
spark = SparkSession.builder.appName("FoodSearchCountWithWatermarking").getOrCreate()

# 소켓을 통한 스트리밍 데이터 수신 설정
# 'localhost'의 '9999' 포트에서 데이터를 수신하여 스트림을 생성합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.format.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.readStream.html
lines = spark._____________._____________("socket").option("host", "localhost").option("port", 9999).load()

# 현재 시간 열 추가 및 단어 분리
# 데이터 수신 시점의 타임스탬프를 추가하여 이벤트 시간 기반 처리가 가능하도록 하고, 
# explode와 split 함수를 사용하여 입력된 텍스트를 단어 단위로 분리합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.current_timestamp.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.explode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.split.html
lines_with_timestamp = lines.withColumn("timestamp", ______________())
foods = lines_with_timestamp.select(______________(split(lines.value, " ")).alias("food"), "timestamp")

# 특정 음식 필터링
# 사용자가 입력한 단어 중 '피자', '치킨', '버거'에 해당하는 경우만 필터링하여 집계에 포함합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.expr.html
tracked_foods = foods.filter(______________("food IN ('피자', '치킨', '버거')"))

# 워터마크 및 윈도우를 적용하여 음식 검색 빈도 집계
# 워터마크 설정 (10분): 10분이 지난 오래된 데이터는 집계에서 제외됩니다.
# 윈도우 설정 (1분): 각 음식별로 1분 고정 윈도우를 설정하여, 윈도우 내의 검색 빈도를 집계합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.withWatermark.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.window.html
foodSearchCounts = tracked_foods.______________("timestamp", "10 minutes") \
    .groupBy("food", ______________("timestamp", "1 minute")).count()

# 현재 시간 기준으로 10분 이내 데이터만 필터링
# outputMode("complete")와 함께 사용할 때, 오래된 데이터가 반복 출력되지 않도록 윈도우 종료 시간이 
# 현재 시간에서 10분 이내인 데이터만 남깁니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.Column.html
filteredCounts = foodSearchCounts.filter(
    col("window.end") >= current_timestamp() - expr("INTERVAL 10 MINUTES")
)

# 실시간 출력 설정
# outputMode("complete"): 매번 전체 집계 결과를 출력합니다.
# truncate 옵션을 비활성화하여 출력 시 내용을 자르지 않습니다.
# trigger(processingTime="5 seconds"): 5초마다 데이터가 갱신됩니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.outputMode.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.writeStream.html
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.DataStreamWriter.trigger.html
query = filteredCounts.______________ \
    .______________("complete") \
    .format("console") \
    .option("truncate", "false") \
    .______________(processingTime="5 seconds") \
    .start()

# 스트리밍 쿼리를 실행하고 프로그램이 종료되지 않도록 유지합니다.
# 참고: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.streaming.StreamingQuery.awaitTermination.html
query.awaitTermination()


24/11/09 11:20:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/11/09 11:20:07 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/11/09 11:20:07 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/11/09 11:20:07 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7c8ab994-6335-4c12-9c1f-d6f32230a31e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/11/09 11:20:07 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+------+-----+
|food|window|count|
+----+------+-----+
+----+------+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+------------------------------------------+-----+
|food|window                                    |count|
+----+------------------------------------------+-----+
|치킨|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|1    |
|피자|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|1    |
|버거|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|1    |
+----+------------------------------------------+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+------------------------------------------+-----+
|food|window                                    |count|
+----+------------------------------------------+-----+
|치킨|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|2    |
|피자|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|5    |
|버거|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|2    |
+----+------------------------------------------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+------------------------------------------+-----+
|food|window                                    |count|
+----+------------------------------------------+-----+
|치킨|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|2    |
|피자|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|9    |
|버거|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|2    |
+----+------------------------------------------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+----+------------------------------------------+-----+
|food|window                                    |count|
+----+------------------------------------------+-----+
|치킨|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|2    |
|피자|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|9    |
|버거|{2024-11-09 11:20:00, 2024-11-09 11:21:00}|2    |
|피자|{2024-11-09 11:21:00, 2024-11-09 11:22:00}|2    |
+----+------------------------------------------+-----+

